In [254]:
#Import 
import pandas as pd
import numpy as np
from scipy import signal
import datetime

In [255]:
#File Infomation
name   = 'yoshiyama'
folder = r'D:\iriwa\デスクトップ\Code\Project_Python\Research\data'
slice_min = 5

In [256]:
#KSS
input_KSS_path = folder + '\\'+ name + '_KSS.xlsx'
input_KSS      = pd.read_excel(input_KSS_path)
print(input_KSS_path)
# print(input_KSS)

output_KSS = pd.DataFrame({'KSS[-]': [input_KSS.iloc[i,1] for i in range(len(input_KSS))for j in range(30)]})
print(output_KSS)

D:\iriwa\デスクトップ\Code\Project_Python\Research\data\yoshiyama_KSS.xlsx
      KSS[-]
0          3
1          3
2          3
3          3
4          3
...      ...
1825       3
1826       3
1827       3
1828       3
1829       3

[1830 rows x 1 columns]


In [257]:
#Monitoring
input_Monitor_A_path = folder + '\\'+ name + '_Monitoring_A.xlsx'
input_Monitor_A      = pd.read_excel(input_Monitor_A_path)
input_Monitor_B_path = folder + '\\'+ name + '_Monitoring_B.xlsx'
input_Monitor_B      = pd.read_excel(input_Monitor_B_path)
input_Monitor_C_path = folder + '\\'+ name + '_Monitoring_C.xlsx'
input_Monitor_C      = pd.read_excel(input_Monitor_C_path)
print(input_Monitor_A_path)
# print(input_Monitor_A)
print(input_Monitor_B_path)
# print(input_Monitor_B)
print(input_Monitor_C_path)
# print(input_Monitor_C)

Monitor_A = pd.DataFrame({'客観評価_A[-]': [input_Monitor_A.iloc[i,1] for i in range(len(input_Monitor_A))for j in range(10)]})
Monitor_B = pd.DataFrame({'客観評価_B[-]': [input_Monitor_B.iloc[i,1] for i in range(len(input_Monitor_B))for j in range(10)]})
Monitor_C = pd.DataFrame({'客観評価_C[-]': [input_Monitor_C.iloc[i,1] for i in range(len(input_Monitor_C))for j in range(10)]})
# print(Monitor_A)
# print(Monitor_B)
# print(Monitor_C)

Monitor_ABC = pd.concat([Monitor_A, Monitor_B, Monitor_C], axis=1)
# print('Monitor_ABC')
# print(Monitor_ABC)


Monitor_mode = Monitor_ABC.fillna(0).mode(axis=1)
NaN = pd.DataFrame([np.nan for i in range(len(Monitor_mode))])
Monitor_mode = pd.concat([Monitor_mode, NaN,NaN],axis=1)
#print(Monitor_mode)
result = [np.nan for i in range(len(Monitor_ABC))]
for i in range(len(Monitor_ABC)):
    #print(Monitor_mode.iloc[i,0])
    if Monitor_mode.iloc[i,0] == 0.0:
        # 0 NaN NaN >> 1 or 2
        if pd.isnull(Monitor_mode.iloc[i,1]):
            result[i] = float(Monitor_ABC.iloc[i,].dropna().values)
        # 0 1 3 >> 2    # 0 1 2 >> 2
        elif Monitor_mode.iloc[i,1] == 1.0:
            result[i] = 2.0
        # 0 2 3 >> 3
        else:
            result[i] = 3.0
    else:
        # 1 NaN NaN >> 1 ...
        if pd.isnull(Monitor_mode.iloc[i,1]):
            result[i] = Monitor_mode.iloc[i,0]
        # 1 2 3 >> 2
        else:
            result[i] = 2.0
    
Monitor_All = pd.DataFrame(result, columns=['客観評価_総合[-]'])

output_Monitoring = pd.concat([Monitor_ABC, Monitor_All], axis=1)
print(output_Monitoring)

D:\iriwa\デスクトップ\Code\Project_Python\Research\data\yoshiyama_Monitoring_A.xlsx
D:\iriwa\デスクトップ\Code\Project_Python\Research\data\yoshiyama_Monitoring_B.xlsx
D:\iriwa\デスクトップ\Code\Project_Python\Research\data\yoshiyama_Monitoring_C.xlsx
      客観評価_A[-]  客観評価_B[-]  客観評価_C[-]  客観評価_総合[-]
0             1          1        NaN         1.0
1             1          1        NaN         1.0
2             1          1        NaN         1.0
3             1          1        NaN         1.0
4             1          1        NaN         1.0
...         ...        ...        ...         ...
1805          1          1        NaN         1.0
1806          1          1        NaN         1.0
1807          1          1        NaN         1.0
1808          1          1        NaN         1.0
1809          1          1        NaN         1.0

[1810 rows x 4 columns]


↓応急処置してるからあとでちゃんとしたの必要

In [258]:
#Eye
input_Eye_path = folder + '\\'+ name + '_Eye.csv'
input_Eye      = pd.read_csv(input_Eye_path,sep=';').dropna()
print(input_Eye_path)
#print(input_Eye[' timestamp '])

Eye_N    = 128
Eye_fs   = int(len(input_Eye)//len(output_Monitoring))
Eye_dt   = 1/Eye_fs

Eye_len  = int(len(input_Eye))
Eye_time = int(Eye_len//Eye_fs)-1

#################################################
pupil_data = pd.DataFrame({'pupil': [i if i>5 else np.nan for i in input_Eye[' pupil diameter']]})
pupil_data = pupil_data.interpolate()

pupil_diameter = pd.DataFrame({'瞳孔径[px]': [pupil_data1.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].mean() for i in range(Eye_time)]})


Eye_wf          = signal.hamming(Eye_N)
pupil_MPF_data  = [np.nan for i in range(Eye_time)]
pupil_list      = [i for i in pupil_data['pupil']]
pupil_spectrum_data =[]
# print(pupil_list)

for i in range(Eye_time):
    pupil_FFT = [pupil_list[j+i*int(Eye_fs)] * Eye_wf[j] for j in range(Eye_N)]
    #高速フーリエ変換
    pupil_F         = np.fft.fft(pupil_FFT)

    pupil_amplitude = np.abs(pupil_F)   #振幅スペクトル(F/N/2)
    pupil_power     = pupil_amplitude ** 2        #パワースペクトル
    pupil_density   = pupil_power/(Eye_fs/Eye_N)     #パワースペクトル密度
    pupil_freq      = np.fft.fftfreq(Eye_N, d=Eye_dt)    #周波数
    pupil_f_l       = int(len(pupil_freq)/2)
    pupil_spectrum_data.append(pupil_power)

    #Func_cal
    pupil_total_fp = 0
    pupil_total_power = 0

    for k in range(pupil_f_l):
        if pupil_freq[k] >= 1. and pupil_freq[k] <= 60.: #0~60Hz
            pupil_total_power += pupil_amplitude[k]
            pupil_total_fp     = pupil_total_fp + (pupil_amplitude[k] * pupil_freq[k])
    pupil_MPF_data[i] = pupil_total_fp / pupil_total_power

pupil_MPF = pd.DataFrame(pupil_MPF_data, columns=['瞳孔径_MPF[Hz]'])
#print(pupil_MPF)

output_pupil = pd.concat([pupil_diameter, pupil_MPF], axis=1)
print(output_pupil)
#################################################
pupil_spectrum = pd.DataFrame(pupil_spectrum_data, columns=pupil_freq)
for i in range(len(pupil_freq),-1,-1):
    if pupil_spectrum.columns[i-1] <= 0:
        pupil_spectrum.drop(columns=pupil_spectrum.columns[i-1], axis=1, inplace=True)
#print(pupil_spectrum)

pupil_spec = pupil_spectrum.values
slice_time = slice_min*60
spec_len = len(pupil_spectrum)
pupil_spec_time = [[np.nan for i in range(int(len(pupil_spectrum.columns)))] for j in range(int(spec_len/slice_time))]
for i in range(int(spec_len/slice_time)):
    for j in range(int(len(pupil_spectrum.columns))):
        pupil_spec_time[i][j]=np.mean(pupil_spec[int(i*slice_time):int((i+1)*slice_time), j])
pupil_index = ['-'.join([str(i*slice_min),str((i+1)*slice_min)]) for i in range(int(spec_len/slice_time))]
pupil_spectrum_time = pd.DataFrame(pupil_spec_time, index=pupil_index,columns=pupil_spectrum.columns)
#print(pupil_spectrum_time)
#################################################
perclos_time = int(Eye_len/Eye_fs)
perclos_data = input_Eye[' eyelids gap '].interpolate()
max_gap = perclos_data.max()
min_gap = perclos_data.min()
perclos_data = pd.DataFrame({'perclos': [i/max_gap for i in perclos_data]})



p_10 = pd.DataFrame({'perclos10': [1 if i<0.4 else 0 for i in perclos_data['perclos']]})
p_20 = pd.DataFrame({'perclos20': [1 if i<0.3 else 0 for i in perclos_data['perclos']]})
p_30 = pd.DataFrame({'perclos30': [1 if i<0.2 else 0 for i in perclos_data['perclos']]})
p_40 = pd.DataFrame({'perclos40': [1 if i<0.1 else 0 for i in perclos_data['perclos']]})
p_50 = pd.DataFrame({'perclos50': [1 if i<0.5 else 0 for i in perclos_data['perclos']]})
p_60 = pd.DataFrame({'perclos60': [1 if i<0.4 else 0 for i in perclos_data['perclos']]})
p_70 = pd.DataFrame({'perclos70': [1 if i<0.3 else 0 for i in perclos_data['perclos']]})
p_80 = pd.DataFrame({'perclos80': [1 if i<0.2 else 0 for i in perclos_data['perclos']]})
p_90 = pd.DataFrame({'perclos90': [1 if i<0.1 else 0 for i in perclos_data['perclos']]})

per_10 = pd.DataFrame({'perclos10[-]': [p_10.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_20 = pd.DataFrame({'perclos20[-]': [p_20.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_30 = pd.DataFrame({'perclos30[-]': [p_30.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_40 = pd.DataFrame({'perclos40[-]': [p_40.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_50 = pd.DataFrame({'perclos50[-]': [p_50.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_60 = pd.DataFrame({'perclos60[-]': [p_60.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_70 = pd.DataFrame({'perclos70[-]': [p_70.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_80 = pd.DataFrame({'perclos80[-]': [p_80.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})
per_90 = pd.DataFrame({'perclos90[-]': [p_90.iloc[i*Eye_fs:i*Eye_fs+Eye_fs,0].sum() for i in range(perclos_time)]})


perclos_10 = per_10.rolling(60).sum()/7200
perclos_20 = per_20.rolling(60).sum()/7200
perclos_30 = per_30.rolling(60).sum()/7200
perclos_40 = per_40.rolling(60).sum()/7200
perclos_50 = per_50.rolling(60).sum()/7200
perclos_60 = per_60.rolling(60).sum()/7200
perclos_70 = per_70.rolling(60).sum()/7200
perclos_80 = per_80.rolling(60).sum()/7200
perclos_90 = per_90.rolling(60).sum()/7200

output_perclos = pd.concat([perclos_10, perclos_20, perclos_30, perclos_40,perclos_50, perclos_60, perclos_70, perclos_80, perclos_90], axis=1)
print(output_perclos)

D:\iriwa\デスクトップ\Code\Project_Python\Research\data\yoshiyama_Eye.csv
        瞳孔径[px]  瞳孔径_MPF[Hz]
0     39.803279    23.819866
1     42.500000    15.852428
2     41.459016    25.691939
3     40.283019    26.645437
4     40.836066    24.886166
...         ...          ...
1811        NaN    15.095683
1812        NaN    18.012488
1813        NaN    16.973276
1814        NaN     4.794554
1815        NaN    16.693532

[1816 rows x 2 columns]
      perclos10[-]  perclos20[-]  perclos30[-]  perclos40[-]  perclos50[-]  \
0              NaN           NaN           NaN           NaN           NaN   
1              NaN           NaN           NaN           NaN           NaN   
2              NaN           NaN           NaN           NaN           NaN   
3              NaN           NaN           NaN           NaN           NaN   
4              NaN           NaN           NaN           NaN           NaN   
...            ...           ...           ...           ...           ...   
1812      0.1

In [259]:
#EEG
input_EEG_path = folder + '\\'+ name + '_EEG.txt'
input_EEG      = pd.read_table(input_EEG_path, header=None, skiprows=6, names=('time','O1','O2'))
print(input_EEG_path)
#print(input_EEG)

EEG_N  = 1024
EEG_dt = 0.001
EEG_fs = 1/EEG_dt
EEG_len = int(len(input_EEG)/EEG_fs)-1
print('%d秒分のデータ'%EEG_len)
#################################################
EEG_filter = signal.firwin(numtaps=16, cutoff=[1,50],pass_zero=False, fs=1/EEG_dt) # Filter (bandpass)
EEG_wf = signal.hamming(EEG_N)

EEG_data = []
EEG_spectrum_data1 = []
EEG_spectrum_data2 = []
for n in range(2):
    filter_data = signal.lfilter(EEG_filter, 1, input_EEG.iloc[:, n+1])
    
    EEG_MPF   = [np.nan for i in range(EEG_len)]
    EEG_alpha = [np.nan for i in range(EEG_len)]
    EEG_beta  = [np.nan for i in range(EEG_len)]
    EEG_theta = [np.nan for i in range(EEG_len)]
    # EEG_delta = [np.nan for i in range(EEG_len)]
    EEG_gamma  = [np.nan for i in range(EEG_len)]

    EEG_alpha1 = [np.nan for i in range(EEG_len)]
    EEG_alpha2 = [np.nan for i in range(EEG_len)]
    EEG_beta1  = [np.nan for i in range(EEG_len)]
    EEG_beta2  = [np.nan for i in range(EEG_len)]
    EEG_gamma1 = [np.nan for i in range(EEG_len)]
    EEG_gamma2 = [np.nan for i in range(EEG_len)]

    for i in range(EEG_len):
        EEG_FFT = [filter_data[j+i*int(EEG_fs)] * EEG_wf[j] for j in range(EEG_N)]

        #高速フーリエ変換
        EEG_F = np.fft.fft(EEG_FFT)
        EEG_amplitude = np.abs(EEG_F)   # Amplitude     # 振幅スペクトル(F/N/2)
        EEG_power = EEG_amplitude ** 2        # Power         # パワースペクトル
        EEG_density = EEG_power/(EEG_fs/EEG_N)     # Power Density # パワースペクトル密度
        EEG_freq = np.fft.fftfreq(EEG_N, d=EEG_dt)
        EEG_f_l = int(len(EEG_freq)/2)
        if n == 0:
            EEG_spectrum_data1.append(EEG_power)
        else:
            EEG_spectrum_data2.append(EEG_power)
        
        #Func_cal
        sum_alpha = 0
        sum_beta  = 0
        sum_theta = 0
        # sum_delta = 0
        sum_gamma = 0
        sum_alpha1 = 0
        sum_alpha2 = 0
        sum_beta1  = 0
        sum_beta2  = 0
        sum_gamma1 =0
        sum_gamma2 =0

        total_fp = 0
        total_power = 0

        for l in range(EEG_f_l):
            if EEG_freq[l] >= 4.0 and EEG_freq[l] <= 50: 
                total_power += EEG_amplitude[l]
                total_fp = total_fp + (EEG_amplitude[l] * EEG_freq[l])
                # if EEG_freq[l] >= 1.0  and EEG_freq[l] <= 4.0:
                #     sum_delta += EEG_amplitude[l]
                if EEG_freq[l] >= 4.0  and EEG_freq[l] <= 8.0:
                     sum_theta += EEG_amplitude[l]
                if EEG_freq[l] >= 8.0  and EEG_freq[l] <= 13.0:
                    sum_alpha  += EEG_amplitude[l]
                if EEG_freq[l] >= 13.0 and EEG_freq[l] <= 30.0:
                    sum_beta   += EEG_amplitude[l]
                if EEG_freq[l] >= 30.0 and EEG_freq[l] <= 50.0:
                    sum_gamma += EEG_amplitude[l]

                if EEG_freq[l] >= 8.0  and EEG_freq[l] <= 10.0:
                    sum_alpha1 += EEG_amplitude[l]
                if EEG_freq[l] >= 10.0  and EEG_freq[l] <= 13.0:
                    sum_alpha2 += EEG_amplitude[l]
                if EEG_freq[l] >= 13.0 and EEG_freq[l] <= 18.0:
                    sum_beta1  += EEG_amplitude[l]
                if EEG_freq[l] >= 18.0 and EEG_freq[l] <= 30.0:
                    sum_beta2  += EEG_amplitude[l]
                if EEG_freq[l] >= 30.0 and EEG_freq[l] <= 40.0:
                    sum_gamma1 += EEG_amplitude[l]
                if EEG_freq[l] >= 40.0 and EEG_freq[l] <= 50.0:
                    sum_gamma2 += EEG_amplitude[l]

        EEG_MPF[i]   = total_fp  / total_power
        EEG_alpha[i] = sum_alpha / total_power 
        EEG_beta[i]  = sum_beta  / total_power
        EEG_theta[i] = sum_theta / total_power
        # EEG_delta[i] = sum_delta / total_power
        EEG_gamma[i] = sum_gamma / total_power

        EEG_alpha1[i] = sum_alpha1 / total_power 
        EEG_alpha2[i] = sum_alpha2 / total_power 
        EEG_beta1[i]  = sum_beta1  / total_power
        EEG_beta2[i]  = sum_beta2  / total_power
        EEG_gamma1[i] = sum_gamma1 / total_power
        EEG_gamma2[i] = sum_gamma2 / total_power
    
    EEG_data.append(EEG_MPF)
    # EEG_data.append(EEG_delta)
    EEG_data.append(EEG_theta)
    EEG_data.append(EEG_alpha)
    EEG_data.append(EEG_beta)
    EEG_data.append(EEG_gamma)

    EEG_data.append(EEG_alpha1)
    EEG_data.append(EEG_alpha2)
    EEG_data.append(EEG_beta1)
    EEG_data.append(EEG_beta2)
    EEG_data.append(EEG_gamma1)
    EEG_data.append(EEG_gamma2)

# output_EEG = pd.DataFrame(EEG_data, index=['O1_MPF[Hz]', 'O1_alpha_ratio[-]', 'O1_beta_ratio[-]', 'O1_theta_ratio[-]', 'O1_delta_ratio[-]',\
#                                             'O2_MPF[Hz]', 'O2_alpha_ratio[-]', 'O2_beta_ratio[-]', 'O2_theta_ratio[-]', 'O2_delta_ratio[-]']).T
EEG_colums = ['O1_MPF[Hz]', 'O1_theta_ratio[-]', 'O1_alpha_ratio[-]', 'O1_beta_ratio[-]', 'O1_gamma_ratio[-]', \
              'O1_Low_alpha_ratio[-]', 'O1_High_alpha_ratio[-]', 'O1_Low_beta_ratio[-]', 'O1_High_beta_ratio[-]', 'O1_Low_gamma_ratio[-]', 'O1_High_gamma_ratio[-]', \
              'O2_MPF[Hz]', 'O2_alpha_ratio[-]', 'O2_beta_ratio[-]', 'O2_theta_ratio[-]', 'O2_delta_ratio[-]',\
              'O2_Low_alpha_ratio[-]', 'O2_High_alpha_ratio[-]', 'O2_Low_beta_ratio[-]', 'O2_High_beta_ratio[-]', 'O2_Low_gamma_ratio[-]', 'O2_High_gamma_ratio[-]']
output_EEG = pd.DataFrame(EEG_data, index=EEG_colums).T
print(output_EEG)
#################################################
EEG_spectrum1 = pd.DataFrame(EEG_spectrum_data1, columns=EEG_freq)
for i in range(len(EEG_freq),-1,-1):
    if EEG_spectrum1.columns[i-1] <= 0 or EEG_spectrum1.columns[i-1] > 50:
        EEG_spectrum1.drop(columns=EEG_spectrum1.columns[i-1], axis=1, inplace=True)
#print(EEG_spectrum1)
EEG_spectrum2 = pd.DataFrame(EEG_spectrum_data2, columns=EEG_freq)
for i in range(len(EEG_freq),-1,-1):
    if EEG_spectrum2.columns[i-1] <= 0 or EEG_spectrum2.columns[i-1] > 50:
        EEG_spectrum2.drop(columns=EEG_spectrum2.columns[i-1], axis=1, inplace=True)
#print(EEG_spectrum1)
slice_time = slice_min*60
EEG_spec_len = len(EEG_spectrum1)
EEG_spec1 = EEG_spectrum1.values
EEG_spec_time1 = [[np.nan for i in range(int(len(EEG_spectrum1.columns)))] for j in range(int(EEG_spec_len/slice_time))]
EEG_spec2 = EEG_spectrum2.values
EEG_spec_time2 = [[np.nan for i in range(int(len(EEG_spectrum2.columns)))] for j in range(int(EEG_spec_len/slice_time))]
for i in range(int(spec_len/slice_time)):
    for j in range(int(len(EEG_spectrum1.columns))):
        EEG_spec_time1[i][j]=np.mean(EEG_spec1[int(i*slice_time):int((i+1)*slice_time), j])
        EEG_spec_time2[i][j]=np.mean(EEG_spec2[int(i*slice_time):int((i+1)*slice_time), j])
EEG_index = ['-'.join([str(i*slice_min),str((i+1)*slice_min)]) for i in range(int(EEG_spec_len/slice_time))]
EEG_spectrum_time1 = pd.DataFrame(EEG_spec_time1, index=EEG_index,columns=EEG_spectrum1.columns)
EEG_spectrum_time2 = pd.DataFrame(EEG_spec_time2, index=EEG_index,columns=EEG_spectrum2.columns)
#################################################

D:\iriwa\デスクトップ\Code\Project_Python\Research\data\yoshiyama_EEG.txt
1845秒分のデータ
      O1_MPF[Hz]  O1_theta_ratio[-]  O1_alpha_ratio[-]  O1_beta_ratio[-]  \
0      24.085921           0.082185           0.230654          0.324976   
1      22.565518           0.103041           0.249748          0.385423   
2      21.438443           0.152553           0.162125          0.441598   
3      24.493523           0.164062           0.138047          0.325892   
4      20.014200           0.108721           0.272257          0.399270   
...          ...                ...                ...               ...   
1840   15.741204           0.280986           0.237277          0.369839   
1841   16.726987           0.421852           0.189732          0.174907   
1842   12.451926           0.496983           0.215378          0.206802   
1843   13.417934           0.391760           0.282328          0.237833   
1844   13.342801           0.462533           0.240370          0.183555   

      O1

In [260]:
#Coupling
output_data1 = pd.concat([output_KSS, output_Monitoring, output_pupil, output_perclos, output_EEG], axis=1)
time = pd.DataFrame({'時間[s]': [i for i in range(len(output_data1))]})
output_data1 = pd.concat([time, output_data1], axis=1)

output_data2 = pd.concat([output_KSS, output_Monitoring, output_pupil, output_perclos, pupil_spectrum], axis=1)
time = pd.DataFrame({'時間[s]': [i for i in range(len(output_data2))]})
output_data2 = pd.concat([time, output_data2], axis=1)

output_data3 = pd.concat([output_KSS, output_Monitoring, output_EEG], axis=1)
time = pd.DataFrame({'時間[s]': [i for i in range(len(output_data3))]})
output_data3 = pd.concat([time, output_data3], axis=1)

output_data4 = pd.concat([pupil_spectrum_time.T,pupil_spectrum.T], axis=1)

output_data5 = pd.concat([EEG_spectrum_time1.T,EEG_spectrum_time2.T], axis=1)

print('出力データ1>>')  #all data
print(output_data1)
print('出力データ2>>')  #Eye data
print(output_data2)
print('出力データ3>>')  #EEG data
print(output_data3)

print('出力データ4>>')  #Eye spec data
print(output_data4)
print('出力データ5>>')  #EEG spec data
print(output_data5)

出力データ1>>
      時間[s]  KSS[-]  客観評価_A[-]  客観評価_B[-]  客観評価_C[-]  客観評価_総合[-]    瞳孔径[px]  \
0         0     3.0        1.0        1.0        NaN         1.0  39.803279   
1         1     3.0        1.0        1.0        NaN         1.0  42.500000   
2         2     3.0        1.0        1.0        NaN         1.0  41.459016   
3         3     3.0        1.0        1.0        NaN         1.0  40.283019   
4         4     3.0        1.0        1.0        NaN         1.0  40.836066   
...     ...     ...        ...        ...        ...         ...        ...   
1840   1840     NaN        NaN        NaN        NaN         NaN        NaN   
1841   1841     NaN        NaN        NaN        NaN         NaN        NaN   
1842   1842     NaN        NaN        NaN        NaN         NaN        NaN   
1843   1843     NaN        NaN        NaN        NaN         NaN        NaN   
1844   1844     NaN        NaN        NaN        NaN         NaN        NaN   

      瞳孔径_MPF[Hz]  perclos10[-]  perclos20

In [261]:
#Output
output_folder_path = 'D:\\iriwa\\デスクトップ\\Code\\Project_Python\\Research'
with pd.ExcelWriter(output_folder_path + '\\' + name +'_datalist.xlsx') as writer:
    output_data1.to_excel(writer, sheet_name='全体('+name+')', index=False)
    output_data2.to_excel(writer, sheet_name='眼球('+name+')', index=False)
    output_data3.to_excel(writer, sheet_name='脳波('+name+')', index=False)
    output_data4.to_excel(writer, sheet_name='瞳孔スペクトル('+name+')')
    output_data5.to_excel(writer, sheet_name='脳波スペクトル('+name+')')
